In [ ]:
import numpy as np
import pandas as pd

from typing import List, Dict, Tuple

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error

from xgboost import XGBRegressor

# ---------- small helpers ----------

# Targets to train on (adjust this list if needed)
TARGET_COLS = [
    "oc_usda.c729_w.pct",
    "c.tot_usda.a622_w.pct",
    "n.tot_usda.a623_w.pct",
    "ph.h2o_usda.a268_index",
    "ph.cacl2_usda.a481_index",
    "cec_usda.a723_cmolc.kg",
    "ec_usda.a364_ds.m",
    "clay.tot_usda.a334_w.pct",
    "sand.tot_usda.c60_w.pct",
    "silt.tot_usda.c62_w.pct",
    "bd_usda.a4_g.cm3",
    "wr.10kPa_usda.a414_w.pct",
    "wr.33kPa_usda.a415_w.pct",
    "wr.1500kPa_usda.a417_w.pct",
    "awc.33.1500kPa_usda.c80_w.frac",
    "fe.ox_usda.a60_w.pct",
    "al.ox_usda.a59_w.pct",
    "fe.dith_usda.a66_w.pct",
    "al.dith_usda.a65_w.pct",
    "p.ext_usda.a1070_mg.kg",
    "k.ext_usda.a1065_mg.kg",
    "mg.ext_usda.a1066_mg.kg",
    "ca.ext_usda.a1059_mg.kg",
    "na.ext_usda.a1068_mg.kg",
]

# Targets to train in log1p-space (transform back with expm1 for metrics/use)
LOG_TRANSFORM_TARGETS = {
    "p.ext_usda.a1070_mg.kg",
    "k.ext_usda.a1065_mg.kg",
    "mg.ext_usda.a1066_mg.kg",
    "ca.ext_usda.a1059_mg.kg",
    "na.ext_usda.a1068_mg.kg",
    "fe.dith_usda.a66_w.pct",
    "al.dith_usda.a65_w.pct",
    "fe.ox_usda.a60_w.pct",
    "al.ox_usda.a59_w.pct",
    "s.ext_mel3_mg.kg",
    "s.tot_usda.a624_w.pct",
    "caco3_usda.a54_w.pct",
}


def snv_transform(X: np.ndarray) -> np.ndarray:
    """
    Apply Standard Normal Variate (SNV) row-wise to spectra.
    X: (n_samples, n_features) numeric array.
    """
    mean = X.mean(axis=1, keepdims=True)
    std = X.std(axis=1, keepdims=True)
    # Avoid division by zero
    std[std == 0] = 1.0
    return (X - mean) / std


def build_feature_matrix_per_sample(
    df: pd.DataFrame,
    extra_numeric_cols: List[str] = None,
    n_pca_components: int = 120,
) -> Tuple[np.ndarray, PCA, SimpleImputer, SimpleImputer, StandardScaler, List[str], List[str]]:
    """
    Build the feature matrix X from the cleaned OSSL dataframe.

    Steps:
    - Select spectral columns: VisNIR (scan_visnir.*_ref) + MIR (scan_mir.*_abs)
    - Impute missing spectral values with column medians
    - Apply SNV (Standard Normal Variate) per row
    - PCA-compress to n_pca_components (or fewer if not enough bands)
    - Optionally add extra numeric columns (imputed + scaled)

    Returns
    -------
    X : np.ndarray
        Feature matrix (n_samples, n_features_total)
    pca : PCA
        Fitted PCA object on SNV spectra.
    spec_imputer : SimpleImputer
        Imputer used on spectral columns.
    num_imputer : SimpleImputer
        Imputer used on extra numeric columns.
    num_scaler : StandardScaler
        Scaler used on extra numeric columns.
    spectral_cols : list of str
        Names of spectral columns used.
    extra_used : list of str
        Extra numeric columns actually included.
    """
    # 1. Spectral columns
    spectral_cols = [
        c for c in df.columns
        if (c.startswith("scan_visnir.") and c.endswith("_ref"))
        or (c.startswith("scan_mir.") and c.endswith("_abs"))
    ]
    if len(spectral_cols) == 0:
        raise ValueError("No spectral columns found (scan_visnir.*_ref / scan_mir.*_abs).")

    spec_imputer = SimpleImputer(strategy="median")
    X_spec = spec_imputer.fit_transform(df[spectral_cols].astype(float))

    # 2. SNV (row-wise)
    spec_mean = X_spec.mean(axis=1, keepdims=True)
    spec_std = X_spec.std(axis=1, keepdims=True)
    spec_std[spec_std == 0] = 1.0
    X_spec_snv = (X_spec - spec_mean) / spec_std

    # 3. PCA
    n_components = min(n_pca_components, X_spec_snv.shape[1])
    pca = PCA(n_components=n_components, random_state=42)
    X_spec_pcs = pca.fit_transform(X_spec_snv)

    # 4. Extra numeric columns (depth, lat/long)
    if extra_numeric_cols is None:
        extra_numeric_cols = [
            "layer.upper.depth_usda_cm",
            "layer.lower.depth_usda_cm",
            "latitude.point_wgs84_dd",
            "longitude.point_wgs84_dd",
        ]

    extra_used = [c for c in extra_numeric_cols if c in df.columns]
    if len(extra_used) > 0:
        num_imputer = SimpleImputer(strategy="median")
        X_num = num_imputer.fit_transform(df[extra_used].astype(float))
        num_scaler = StandardScaler()
        X_num_scaled = num_scaler.fit_transform(X_num)
        X = np.hstack([X_spec_pcs, X_num_scaled])
    else:
        num_imputer = None
        num_scaler = None
        X = X_spec_pcs

    return X, pca, spec_imputer, num_imputer, num_scaler, spectral_cols, extra_used



def select_targets(df: pd.DataFrame) -> List[str]:
    """
    Choose a sensible default list of target soil properties based on
    what we discussed (SOC, texture, pH, CEC, WR, etc.) that exist in df.
    """
    candidate_targets = [
        # Chemistry
        "oc_usda.c729_w.pct",
        "c.tot_usda.a622_w.pct",
        "n.tot_usda.a623_w.pct",
        "ph.h2o_usda.a268_index",
        "ph.cacl2_usda.a481_index",
        "cec_usda.a723_cmolc.kg",
        "ec_usda.a364_ds.m",
        # Texture & physical
        "clay.tot_usda.a334_w.pct",
        "sand.tot_usda.c60_w.pct",
        "silt.tot_usda.c62_w.pct",
        "bd_usda.a4_g.cm3",
        # Water retention
        "wr.10kPa_usda.a414_w.pct",
        "wr.33kPa_usda.a415_w.pct",
        "wr.1500kPa_usda.a417_w.pct",
        "awc.33.1500kPa_usda.c80_w.frac",
        # Fe/Al oxides
        "fe.ox_usda.a60_w.pct",
        "al.ox_usda.a59_w.pct",
        "fe.dith_usda.a66_w.pct",
        "al.dith_usda.a65_w.pct",
        # Nutrients (weaker but useful)
        "p.ext_usda.a1070_mg.kg",
        "k.ext_usda.a1065_mg.kg",
        "mg.ext_usda.a1066_mg.kg",
        "ca.ext_usda.a1059_mg.kg",
        "na.ext_usda.a1068_mg.kg",
    ]

    target_cols = [c for c in candidate_targets if c in df.columns]
    if not target_cols:
        raise ValueError("No candidate target columns found in DataFrame.")
    return target_cols


def learning_rate_decay(initial_lr=0.05, decay=0.95):
    """
    Returns a function suitable for XGBoost's LearningRateScheduler callback.
    lr(t) = initial_lr * decay^t
    """
    def _lr(step: int) -> float:
        return initial_lr * (decay ** step)
    return _lr


def evaluate_multioutput(
    y_true_trans: np.ndarray,
    y_pred_trans: np.ndarray,
    target_names: List[str],
    log_transform_targets: set,
):
    """
    Print per-target R2 and RMSE on the ORIGINAL scale.

    Parameters
    ----------
    y_true_trans : array, shape (n_samples, n_targets)
        Ground truth in the same (possibly log-transformed) space used for training.
    y_pred_trans : array, shape (n_samples, n_targets)
        Predictions in the same (possibly log-transformed) space.
    target_names : list of str
        Column names corresponding to each target index.
    log_transform_targets : set of str
        Names of targets that were trained in log1p space and must be
        back-transformed with expm1 for metric computation.
    """
    r2_list = []
    rmse_list = []

    for i, name in enumerate(target_names):
        yt = y_true_trans[:, i]
        yp = y_pred_trans[:, i]

        # Back to original scale if this target was log-transformed
        if name in log_transform_targets:
            yt = np.expm1(yt)
            yp = np.expm1(yp)

        r2 = r2_score(yt, yp)
        rmse = mean_squared_error(yt, yp, squared=False)
        r2_list.append(r2)
        rmse_list.append(rmse)
        print(f"  {name:30s}  R2 = {r2:6.3f}   RMSE = {rmse:8.3f}")

    print("\n  ---- Averages ----")
    print(f"  Mean R2   : {np.nanmean(r2_list):6.3f}")
    print(f"  Mean RMSE : {np.nanmean(rmse_list):8.3f}")


# ---------- main pipeline ----------

def train_per_target_models(
    csv_path: str,
    target_cols: List[str] = TARGET_COLS,
    log_transform_targets: set = LOG_TRANSFORM_TARGETS,
    test_size: float = 0.2,
    n_splits_cv: int = 5,
    n_pca_components: int = 120,
    min_rows_required: int = 200,
    use_gpu: bool = True,
):
    """
    OSSL-style training: one XGBoost model per soil property, each using all rows
    where that property is non-missing. Uses shared SNV+PCA spectral features.

    Returns a dict with:
        - models:  dict[target] -> fitted XGBRegressor
        - metrics: dict[target] -> CV/test metrics
        - pca, spec_imputer, num_imputer, num_scaler
        - spectral_cols, extra_used
    """
    df = pd.read_csv(csv_path, low_memory=False)
    print(f"Loaded cleaned dataset: {df.shape} (rows, columns)")

    # Build shared feature matrix
    X, pca, spec_imputer, num_imputer, num_scaler, spec_cols, extra_used = build_feature_matrix_per_sample(
        df,
        n_pca_components=n_pca_components,
    )
    print(f"Feature matrix shape: {X.shape}")
    print(f"  Spectral columns used: {len(spec_cols)}")
    print(f"  Extra numeric columns used: {extra_used}")

    models: Dict[str, XGBRegressor] = {}
    metrics: Dict[str, dict] = {}

    # ---- XGBoost factory ----
    def make_xgb() -> XGBRegressor:
        params = dict(
            n_estimators=600,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            random_state=42,
            n_jobs=-1,
        )
        if use_gpu:
            params.update(
                tree_method="hist",  # with device="cuda" this uses GPU in XGB >= 2.0
                device="cuda",
            )
        else:
            params.update(
                tree_method="hist",
                device="cpu",
            )
        return XGBRegressor(**params)

    for target in target_cols:
        if target not in df.columns:
            print(f"\n[SKIP] Target {target} not found in dataframe columns.")
            continue

        y = df[target].to_numpy(dtype=float)
        mask = ~np.isnan(y)
        n_available = int(mask.sum())

        print(f"\n=== Target: {target} | available samples: {n_available} ===")

        if n_available < min_rows_required:
            print(f"  [SKIP] Only {n_available} non-missing rows (< {min_rows_required}).")
            continue

        X_t = X[mask]
        y_t = y[mask]

        is_log = target in log_transform_targets
        if is_log:
            y_t = np.log1p(y_t)

        X_trainval, X_test, y_trainval, y_test = train_test_split(
            X_t, y_t, test_size=test_size, random_state=42
        )

        # CV
        kf = KFold(n_splits=n_splits_cv, shuffle=True, random_state=42)
        cv_r2_list, cv_rmse_list = [], []

        for fold, (tr_idx, val_idx) in enumerate(kf.split(X_trainval), start=1):
            X_tr, X_val = X_trainval[tr_idx], X_trainval[val_idx]
            y_tr, y_val = y_trainval[tr_idx], y_trainval[val_idx]

            model = make_xgb()
            model.fit(X_tr, y_tr)

            y_val_pred = model.predict(X_val)

            if is_log:
                y_val_true_orig = np.expm1(y_val)
                y_val_pred_orig = np.expm1(y_val_pred)
            else:
                y_val_true_orig = y_val
                y_val_pred_orig = y_val_pred

            r2 = r2_score(y_val_true_orig, y_val_pred_orig)
            rmse = mean_squared_error(y_val_true_orig, y_val_pred_orig, squared=False)
            cv_r2_list.append(r2)
            cv_rmse_list.append(rmse)

        cv_r2_mean = float(np.mean(cv_r2_list))
        cv_rmse_mean = float(np.mean(cv_rmse_list))
        print(f"  CV (mean over {n_splits_cv} folds): R2 = {cv_r2_mean:.3f}, RMSE = {cv_rmse_mean:.3f}")

        # Final model on train+val
        final_model = make_xgb()
        final_model.fit(X_trainval, y_trainval)

        # Test
        y_test_pred = final_model.predict(X_test)
        if is_log:
            y_test_true_orig = np.expm1(y_test)
            y_test_pred_orig = np.expm1(y_test_pred)
        else:
            y_test_true_orig = y_test
            y_test_pred_orig = y_test_pred

        test_r2 = float(r2_score(y_test_true_orig, y_test_pred_orig))
        test_rmse = float(mean_squared_error(y_test_true_orig, y_test_pred_orig, squared=False))
        print(f"  Test: R2 = {test_r2:.3f}, RMSE = {test_rmse:.3f}")

        models[target] = final_model
        metrics[target] = {
            "n_samples": n_available,
            "cv_r2_mean": cv_r2_mean,
            "cv_rmse_mean": cv_rmse_mean,
            "test_r2": test_r2,
            "test_rmse": test_rmse,
            "log_transformed": is_log,
        }

    return {
        "models": models,
        "metrics": metrics,
        "pca": pca,
        "spec_imputer": spec_imputer,
        "num_imputer": num_imputer,
        "num_scaler": num_scaler,
        "spectral_cols": spec_cols,
        "extra_used": extra_used,
    }


In [ ]:
results = train_per_target_models(
    "/kaggle/input/ossl-cleaned/ossl_cleaned.csv",
    use_gpu=True,
)

models = results["models"]
metrics = results["metrics"]
pca = results["pca"]
spec_imputer = results["spec_imputer"]
num_imputer = results["num_imputer"]
num_scaler = results["num_scaler"]
spectral_cols = results["spectral_cols"]
extra_cols = results["extra_used"]


In [ ]:
import joblib

joblib.dump(
    {
        "models": models,
        "metrics": metrics,
        "pca": pca,
        "spec_imputer": spec_imputer,
        "num_imputer": num_imputer,
        "num_scaler": num_scaler,
        "spectral_cols": spectral_cols,
        "extra_cols": extra_cols,
        "log_transform_targets": list(LOG_TRANSFORM_TARGETS),
    },
    "ossl_per_target_pipeline.pkl",
)
